<a href="https://colab.research.google.com/github/bhaweshiitk/20.C51_project/blob/main/Baseline_Models_20_C51_Graduate_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import matplotlib
import torch
from torch import nn
from torch.nn import functional as F
from sklearn.model_selection import train_test_split
! pip install Bio
!pip install umap-learn
from Bio import SeqIO
import umap

     |████████████████████████████████| 269 kB 8.0 MB/s 
     |████████████████████████████████| 2.3 MB 25.8 MB/s 
     |████████████████████████████████| 88 kB 3.8 MB/s 
     |████████████████████████████████| 1.1 MB 16.4 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=c19256c6db7349efb09ec4a9964e9429af6542a95a75234875990a987d19f94e
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.6-py3-none-any.whl size=53943 sha256=38d0bdb513e3f657b39d0137a381dfab8684c9f9ca5ed2eb7e9f3ba0369d9e54
  Stored in directory: /root/.cache/pip/wheels/03/f1/56/f80d72741e400345b5a5b50ec3d929aca581bf45e0225d5c50
Successfully built umap-learn pynndescent


In [ ]:
# RNN, Hybrid RNN-CNN
# Linear regression, MLP, Random Forest
# Hyperparam tuning 
# Distance between train and test set

In [2]:
# Import data
! wget https://raw.githubusercontent.com/bhaweshiitk/3.C51_project/main/megares_modified_annotations_v2.00.csv
! wget https://raw.githubusercontent.com/bhaweshiitk/3.C51_project/main/megares_modified_database_v2.00.fasta

--2022-05-08 05:54:53--  https://raw.githubusercontent.com/bhaweshiitk/3.C51_project/main/megares_modified_annotations_v2.00.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 948917 (927K) [text/plain]
Saving to: ‘megares_modified_annotations_v2.00.csv’

megares_modified_an 100%[===================>] 926.68K  --.-KB/s    in 0.02s   

2022-05-08 05:54:54 (46.8 MB/s) - ‘megares_modified_annotations_v2.00.csv’ saved [948917/948917]

--2022-05-08 05:54:54--  https://raw.githubusercontent.com/bhaweshiitk/3.C51_project/main/megares_modified_database_v2.00.fasta
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.13

In [3]:
# Process data
train_annotations = pd.read_csv("./megares_modified_annotations_v2.00.csv")
train_annotations.shape

(7868, 5)

In [4]:
train_annotations.head()

,header,type,class,mechanism,group
0,MEG_1|Drugs|Aminoglycosides|Aminoglycoside-res...,Drugs,Aminoglycosides,Aminoglycoside-resistant 16S ribosomal subunit...,A16S
1,MEG_2|Drugs|Aminoglycosides|Aminoglycoside-res...,Drugs,Aminoglycosides,Aminoglycoside-resistant 16S ribosomal subunit...,A16S
2,MEG_3|Drugs|Aminoglycosides|Aminoglycoside-res...,Drugs,Aminoglycosides,Aminoglycoside-resistant 16S ribosomal subunit...,A16S
3,MEG_4|Drugs|Aminoglycosides|Aminoglycoside-res...,Drugs,Aminoglycosides,Aminoglycoside-resistant 16S ribosomal subunit...,A16S
4,MEG_5|Drugs|Aminoglycosides|Aminoglycoside-res...,Drugs,Aminoglycosides,Aminoglycoside-resistant 16S ribosomal subunit...,A16S


In [5]:
train_annotations['Seq_name'] = train_annotations['header'].str.split('|').str[0:2].apply('|'.join)
train_annotations['mechanism'] = train_annotations['header'].str.split('|').str[3]

In [6]:
fasta_sequences = SeqIO.parse(open("./megares_modified_database_v2.00.fasta"),'fasta')
sequence_dict = {}
for fasta in fasta_sequences:
    name, sequence = fasta.id, str(fasta.seq)
    name = "|".join((name.split("|", 2))[0:2])
    sequence_dict[name] = sequence
sequence_df = pd.DataFrame(sequence_dict.items(), columns=['Seq_name', 'Sequence'])
sequence_df.shape

(7868, 2)

In [7]:
sequence_df.head()

,Seq_name,Sequence
0,MEG_1|Drugs,AGAATTTGATCTTGGTTCAGATTGAACGCTGGCGGCGTGGATGAGG...
1,MEG_2|Drugs,AGGTAAGGAGGTGATCCAACCGCAGGTTCCCCTACGGTTACCTTGT...
2,MEG_3|Drugs,TTTTGTTTGGAGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGC...
3,MEG_4|Drugs,AGAGATTGAACATAAGAGTTTGATCCTGGCTCAGATTGAACGCTGG...
4,MEG_5|Drugs,AGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGCGTGCTTAACA...


In [8]:
df = pd.merge(train_annotations[['Seq_name', 'type', 'class','mechanism','group']], 
              sequence_df, how = "inner", on = ["Seq_name"])
df.shape

(7868, 6)

In [9]:
df.head()

,Seq_name,type,class,mechanism,group,Sequence
0,MEG_1|Drugs,Drugs,Aminoglycosides,Aminoglycoside-resistant_16S_ribosomal_subunit...,A16S,AGAATTTGATCTTGGTTCAGATTGAACGCTGGCGGCGTGGATGAGG...
1,MEG_2|Drugs,Drugs,Aminoglycosides,Aminoglycoside-resistant_16S_ribosomal_subunit...,A16S,AGGTAAGGAGGTGATCCAACCGCAGGTTCCCCTACGGTTACCTTGT...
2,MEG_3|Drugs,Drugs,Aminoglycosides,Aminoglycoside-resistant_16S_ribosomal_subunit...,A16S,TTTTGTTTGGAGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGC...
3,MEG_4|Drugs,Drugs,Aminoglycosides,Aminoglycoside-resistant_16S_ribosomal_subunit...,A16S,AGAGATTGAACATAAGAGTTTGATCCTGGCTCAGATTGAACGCTGG...
4,MEG_5|Drugs,Drugs,Aminoglycosides,Aminoglycoside-resistant_16S_ribosomal_subunit...,A16S,AGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGCGTGCTTAACA...


In [10]:
df["class"].value_counts()

betalactams                                        3911
Aminoglycosides                                     773
Drug and biocide resistance                         363
Glycopeptides                                       299
Fluoroquinolones                                    296
Tetracyclines                                       294
MLS                                                 270
Multi-metal resistance                              226
Phenicol                                            145
Trimethoprim                                        135
Copper resistance                                   117
Multi-drug resistance                               113
Sulfonamides                                        103
Multi-biocide resistance                             98
Lipopeptides                                         95
Rifampin                                             70
Fosfomycin                                           62
Mercury resistance                              

In [11]:
df["mechanism"].value_counts()

Class_A_betalactamases                 1580
Class_C_betalactamases                  950
Class_D_betalactamases                  900
Class_B_betalactamases                  410
Aminoglycoside_N-acetyltransferases     359
                                       ... 
Tetracenomycin_MFS_efflux_pump            1
Tunicamycin_resistance_protein            1
Chromate_ABC_efflux_pump                  1
Biguanide_cation_efflux                   1
Sulfonamide_monooxygenase                 1
Name: mechanism, Length: 219, dtype: int64

In [12]:
label_1 = ['Aminoglycosides', 'betalactams', 'Glycopeptides', 'Tetracyclines', 'MLS', 'Fluoroquinolones', 'Trimethoprim',
           'Phenicol', 'Sulfonamides', 'Sulfonamides', 'Lipopeptides']

In [13]:
df['labels'] = df["class"]
df.loc[~df['labels'].isin(label_1),'labels'] = "other"

In [14]:
df.labels.value_counts()

betalactams         3911
other               1547
Aminoglycosides      773
Glycopeptides        299
Fluoroquinolones     296
Tetracyclines        294
MLS                  270
Phenicol             145
Trimethoprim         135
Sulfonamides         103
Lipopeptides          95
Name: labels, dtype: int64

In [31]:
X = df.Sequence.values
y = df.labels.values
print(X.shape, y.shape)
index_to_remove = []
for (i, seq) in enumerate(X):
  if set(seq.upper())!={"A", "T", "C", "G"}:
    index_to_remove.append(i)
index_to_remove
X = np.delete(X, index_to_remove, axis=0)
y = np.delete(y, index_to_remove, axis=0)
print(X.shape, y.shape)


(7868,) (7868,)
(7819,) (7819,)


In [32]:
code = {'A': 1, 'C': 2, 'G': 3, 'T': 4}
label_code_1 = {"betalactams":0, "Aminoglycosides":1, "Glycopeptides":2, "Fluoroquinolones":3, "Tetracyclines":4, "MLS":5,
                 'Trimethoprim':6,'Phenicol':7,  'Sulfonamides':8, 'Lipopeptides':9,"other":10}


In [33]:
def encode_sklearn(seq):
  code1 = {'A': [1, 0, 0, 0], 'C': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'T': [0, 0, 0, 1]}

  enc_seq = []
  for x in seq.upper():
    enc_seq += code1[x]
  if len(enc_seq)<2000:
    enc_seq = enc_seq + [0, 0, 0, 0]*(2000-len(enc_seq))
  return enc_seq[0:2000]


In [34]:
X = np.array([encode_sklearn(elem) for elem in X])
y = np.array([label_code_1[label] for label in y])
X.shape, y.shape

((7819, 2000), (7819,))

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1) 
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1) 

In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier


## Logistic Regression

In [ ]:
model_lr = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty = "l2")
model_lr.fit(X_train, y_train)
ypred = model_lr.predict(X_test)

In [62]:
print("Logistic Regression Accuracy is : %.3f" %((ypred==y_test).mean()))

Logistic Regression Accuracy is : 0.877


## Random Forest

In [65]:
classifier = RandomForestClassifier(random_state = 42)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print("Random Forest Accuracy is : %.3f" %((y_pred==y_test).mean()))

Random Forest Accuracy is : 0.876


## MLP

In [68]:
classifier = MLPClassifier(hidden_layer_sizes=(512, 64), max_iter=300,activation = 'relu',solver='adam',random_state=1)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
print("MLP Accuracy is : %.3f" %((y_pred==y_test).mean()))

MLP Accuracy is : 0.864


# Mechanism of Antibiotic Resistance Prediction

In [69]:
df["labels_2"] = "other"
df.loc[(df['mechanism'].str.contains("nucleotidyltransferases")),'labels_2'] = "nucleotidyltransferases"
df.loc[(df['mechanism'].str.contains("methyltransferases")),'labels_2'] = "methyltransferases"
df.loc[(df['mechanism'].str.contains("phosphotransferases")),'labels_2'] = "phosphotransferases"
df.loc[(df['mechanism'].str.contains("acetyltransferases")),'labels_2'] = "acetyltransferases"
df.loc[(df['mechanism'].str.contains("Quinolone_resistance_protein_Qnr")),'labels_2'] = "Quinolone_resistance_protein_Qnr"
df.loc[(df['mechanism'].str.contains("Dihydrofolate_reductase")),'labels_2'] = "Dihydrofolate_reductase"
df.loc[(df['mechanism'].str.contains("Tetracycline_resistance_ribosomal_protection_proteins")),
       'labels_2'] = "Tetracycline_resistance_ribosomal_protection_proteins"
df.loc[(df['mechanism'].str.contains("efflux_pump")),'labels_2'] = "efflux_pump"
df.loc[(df['mechanism'].str.contains("Fluoroquinolone-resistant_DNA_topoisomerases")),'labels_2'] = "Fluoroquinolone-resistant_DNA_topoisomerases"
df.loc[(df['mechanism'] == "Class_A_betalactamases"),'labels_2'] = "Class_A_betalactamases"
df.loc[(df['mechanism'] == "Class_B_betalactamases"),'labels_2'] = "Class_B_betalactamases"
df.loc[(df['mechanism'] == "Class_C_betalactamases"),'labels_2'] = "Class_C_betalactamases"
df.loc[(df['mechanism'] == "Class_D_betalactamases"),'labels_2'] = "Class_D_betalactamases"
label_code_2 = {}
labels_2 = ["nucleotidyltransferases", "methyltransferases", "phosphotransferases",
            "acetyltransferases", "Quinolone_resistance_protein_Qnr", "Dihydrofolate_reductase", "efflux_pump",
            "Tetracycline_resistance_ribosomal_protection_proteins", "Fluoroquinolone-resistant_DNA_topoisomerases",
            "Class_A_betalactamases", "Class_B_betalactamases", "Class_C_betalactamases","Class_D_betalactamases", "other"]
for (i, elem) in enumerate(labels_2):
  label_code_2[elem] = i

In [70]:
df.labels_2.value_counts()

other                                                    1642
Class_A_betalactamases                                   1580
Class_C_betalactamases                                    950
Class_D_betalactamases                                    900
efflux_pump                                               826
acetyltransferases                                        445
Class_B_betalactamases                                    410
nucleotidyltransferases                                   296
Quinolone_resistance_protein_Qnr                          173
methyltransferases                                        145
Dihydrofolate_reductase                                   135
Tetracycline_resistance_ribosomal_protection_proteins     134
phosphotransferases                                       122
Fluoroquinolone-resistant_DNA_topoisomerases              110
Name: labels_2, dtype: int64

In [73]:
X = df.Sequence.values
y = df.labels_2.values
print(X.shape, y.shape)
index_to_remove = []
for (i, seq) in enumerate(X):
  if set(seq.upper())!={"A", "T", "C", "G"}:
    index_to_remove.append(i)
index_to_remove
X = np.delete(X, index_to_remove, axis=0)
y = np.delete(y, index_to_remove, axis=0)

print(X.shape,y.shape)

(7868,) (7868,)
(7819,) (7819,)


In [74]:
X = np.array([encode_sklearn(elem) for elem in X])
y = np.array([label_code_2[label] for label in y])
X.shape, y.shape

((7819, 2000), (7819,))

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 

## Logistic Regression

In [76]:
model_lr = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty = "l2")
model_lr.fit(X_train, y_train)
ypred = model_lr.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [77]:
print("Logistic Regression Accuracy is : %.3f" %((ypred==y_test).mean()))

Logistic Regression Accuracy is : 0.864


## Random Forest

In [78]:
classifier = RandomForestClassifier(random_state = 42)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print("Random Forest Accuracy is : %.3f" %((y_pred==y_test).mean()))

Random Forest Accuracy is : 0.863


## MLP

In [79]:
classifier = MLPClassifier(hidden_layer_sizes=(512, 64), max_iter=300,activation = 'relu',solver='adam',random_state=1)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
print("MLP Accuracy is : %.3f" %((y_pred==y_test).mean()))

MLP Accuracy is : 0.867
